# Implement Cycle GAN for medical images segmentation

In [2]:
import tensorflow as tf
import tf_slim as slim
from collections import namedtuple
import time
from glob import glob
import numpy as np
import os

In [3]:
LR=0.0002
EPOCH_STEP=100
LOAD_SIZE=286
FINE_SIZE=256
MAX_SIZE=50
PRINT_FREQ=100
SAMPLE_DIR='./sample'
CHECKPOINT_DIR='./checkpoint'
TEST_DIR='./test'
SAVE_FREQ=1000
WHICH_DIRECTION='AtoB'
CONTINUE_TRAIN=True

In [4]:
def instance_norm(input, name="instance_norm"):
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        depth = input.get_shape()[3]
        scale = tf.compat.v1.get_variable("scale", [depth], initializer=tf.compat.v1.random_normal_initializer(1.0, 0.02, dtype=tf.float32))
        offset = tf.compat.v1.get_variable("offset", [depth], initializer=tf.compat.v1.constant_initializer(0.0))
        mean, variance = tf.compat.v1.nn.moments(input, axes=[1,2], keep_dims=True)
        epsilon = 1e-5
        inv = tf.compat.v1.rsqrt(variance + epsilon)
        normalized = (input-mean)*inv
        return scale*normalized + offset

In [5]:
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

In [6]:
def conv2d(input_, output_dim, ks=4, s=2, stddev=0.02, padding='SAME', name="conv2d"):
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        return slim.conv2d(input_, output_dim, ks, s, padding=padding, activation_fn=None,
                            weights_initializer=tf.compat.v1.truncated_normal_initializer(stddev=stddev),
                            biases_initializer=None)

In [7]:
def deconv2d(input_, output_dim, ks=4, s=2, stddev=0.02, name="deconv2d"):
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        return slim.conv2d_transpose(input_, output_dim, ks, s, padding='SAME', activation_fn=None,
                                    weights_initializer=tf.compat.v1.truncated_normal_initializer(stddev=stddev),
                                    biases_initializer=None)

In [8]:
def discriminator(image, options, reuse=False, name="discriminator"):

    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        # image is 256 x 256 x input_c_dim

        h0 = lrelu(conv2d(image, options.df_dim, name='d_h0_conv'))
        # h0 is (128 x 128 x self.df_dim)
        h1 = lrelu(instance_norm(conv2d(h0, options.df_dim*2, name='d_h1_conv'), 'd_bn1'))
        # h1 is (64 x 64 x self.df_dim*2)
        h2 = lrelu(instance_norm(conv2d(h1, options.df_dim*4, name='d_h2_conv'), 'd_bn2'))
        # h2 is (32x 32 x self.df_dim*4)
        h3 = lrelu(instance_norm(conv2d(h2, options.df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
        # h3 is (32 x 32 x self.df_dim*8)
        h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
        # h4 is (32 x 32 x 1)
        return h4

In [9]:
def generator_resnet(image, options, reuse=False, name="generator"):

    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        # image is 256 x 256 x input_c_dim

        def residule_block(x, dim, ks=3, s=1, name='res'):
            p = int((ks - 1) / 2)
            y = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
            y = instance_norm(conv2d(y, dim, ks, s, padding='VALID', name=name+'_c1'), name+'_bn1')
            y = tf.pad(tf.nn.relu(y), [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
            y = instance_norm(conv2d(y, dim, ks, s, padding='VALID', name=name+'_c2'), name+'_bn2')
            return y + x

        # Justin Johnson's model from https://github.com/jcjohnson/fast-neural-style/
        # The network with 9 blocks consists of: c7s1-32, d64, d128, R128, R128, R128,
        # R128, R128, R128, R128, R128, R128, u64, u32, c7s1-3
        c0 = tf.pad(image, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
        c1 = tf.nn.relu(instance_norm(conv2d(c0, options.gf_dim, 7, 1, padding='VALID', name='g_e1_c'), 'g_e1_bn'))
        c2 = tf.nn.relu(instance_norm(conv2d(c1, options.gf_dim*2, 3, 2, name='g_e2_c'), 'g_e2_bn'))
        c3 = tf.nn.relu(instance_norm(conv2d(c2, options.gf_dim*4, 3, 2, name='g_e3_c'), 'g_e3_bn'))
        # define G network with 9 resnet blocks
        r1 = residule_block(c3, options.gf_dim*4, name='g_r1')
        r2 = residule_block(r1, options.gf_dim*4, name='g_r2')
        r3 = residule_block(r2, options.gf_dim*4, name='g_r3')
        r4 = residule_block(r3, options.gf_dim*4, name='g_r4')
        r5 = residule_block(r4, options.gf_dim*4, name='g_r5')
        r6 = residule_block(r5, options.gf_dim*4, name='g_r6')
        r7 = residule_block(r6, options.gf_dim*4, name='g_r7')
        r8 = residule_block(r7, options.gf_dim*4, name='g_r8')
        r9 = residule_block(r8, options.gf_dim*4, name='g_r9')

        d1 = deconv2d(r9, options.gf_dim*2, 3, 2, name='g_d1_dc')
        d1 = tf.nn.relu(instance_norm(d1, 'g_d1_bn'))
        d2 = deconv2d(d1, options.gf_dim, 3, 2, name='g_d2_dc')
        d2 = tf.nn.relu(instance_norm(d2, 'g_d2_bn'))
        d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")
        pred = tf.nn.tanh(conv2d(d2, options.output_c_dim, 7, 1, padding='VALID', name='g_pred_c'))

        return pred

In [10]:
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

In [11]:
def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

In [12]:
from imageio import imread as _imread
from imageio import imwrite
import scipy.misc
from skimage.transform import resize

In [13]:
def inverse_transform(images):
    return (images+1.)/2.

In [14]:
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

In [15]:
def imsave(images, size, path):
    return imwrite(path, merge(images, size))

In [16]:
def save_images(images, size, image_path):
    imsave(inverse_transform(images), size, image_path)

In [17]:
def imread(path, is_grayscale = False):
    if (is_grayscale):
        return _imread(path, flatten=True).astype(np.float)
    else:
        return _imread(path, pilmode='RGB').astype(np.float)

In [18]:
import copy

In [19]:
class ImagePool(object):
    def __init__(self, maxsize=50):
        self.maxsize = maxsize
        self.num_img = 0
        self.images = []

    def __call__(self, image):
        if self.maxsize <= 0:
            return image
        if self.num_img < self.maxsize:
            self.images.append(image)
            self.num_img += 1
            return image
        if np.random.rand() > 0.5:
            idx = int(np.random.rand()*self.maxsize)
            tmp1 = copy.copy(self.images[idx])[0]
            self.images[idx][0] = image[0]
            idx = int(np.random.rand()*self.maxsize)
            tmp2 = copy.copy(self.images[idx])[1]
            self.images[idx][1] = image[1]
            return [tmp1, tmp2]
        
        return image

In [20]:
def load_train_data(image_path, load_size=286, fine_size=256, is_testing=False):
    img_A = imread(image_path[0])
    img_B = imread(image_path[1])
    if not is_testing:
        img_A = resize(img_A, (load_size, load_size))
        img_B = resize(img_B, (load_size, load_size))
        h1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        w1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        img_A = img_A[h1:h1+fine_size, w1:w1+fine_size]
        img_B = img_B[h1:h1+fine_size, w1:w1+fine_size]

        if np.random.random() > 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)
    else:
        img_A = resize(img_A, (fine_size, fine_size))
        img_B = resize(img_B, (fine_size, fine_size))

    img_A = img_A/127.5 - 1.
    img_B = img_B/127.5 - 1.

    img_AB = np.concatenate((img_A, img_B), axis=2)
    # img_AB shape: (fine_size, fine_size, input_c_dim + output_c_dim)
    return img_AB

In [21]:
def load_test_data(image_path, fine_size=256):
    img = imread(image_path)
    img = resize(img, (fine_size, fine_size))
    img = img/127.5 - 1
    return img

In [22]:
class cyclegan(object):
    def __init__(self, sess):
        self.sess = sess
        self.batch_size = 1
        self.image_size = FINE_SIZE
        self.input_c_dim = 3
        self.output_c_dim = 3
        self.L1_lambda = 10.0
        self.dataset_dir = 'med-image'
        
        self.discriminator = discriminator
        self.generator = generator_resnet
        self.criterionGAN = mae_criterion
        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size \
                              gf_dim df_dim output_c_dim is_training')
        self.options = OPTIONS._make((self.batch_size, self.image_size,
                                      64, 64, self.output_c_dim,
                                      True))

        self._build_model()
        self.saver = tf.compat.v1.train.Saver()
        self.pool = ImagePool(MAX_SIZE)

In [23]:
?tf.compat.v1.train.Saver().save

Object `tf.compat.v1.train.Saver().save` not found.


In [24]:
class cyclegan(cyclegan):
    def _build_model(self):
        self.real_data = tf.compat.v1.placeholder(tf.float32,
                                        [None, self.image_size, self.image_size,
                                         self.input_c_dim + self.output_c_dim],
                                        name='real_A_and_B_images')
        self.real_A = self.real_data[:, :, :, :self.input_c_dim]
        self.real_B = self.real_data[:, :, :, self.input_c_dim:self.input_c_dim + self.output_c_dim]
        
        self.fake_A = self.generator(self.real_B, self.options, True, name="generatorB2A")
        self.fake_B = self.generator(self.real_A, self.options, False, name="generatorA2B")
        self.fake_A_ = self.generator(self.fake_B, self.options, False, name="generatorB2A")
        self.fake_B_ = self.generator(self.fake_A, self.options, True, name="generatorA2B")
        
        self.DA_fake = self.discriminator(self.fake_A, self.options, reuse=False, name="discriminatorA")
        self.DB_fake = self.discriminator(self.fake_B, self.options, reuse=False, name="discriminatorB")
        
        self.g_loss_a2b = self.criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)
        self.g_loss_b2a = self.criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)
        
        self.g_loss = self.criterionGAN(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.criterionGAN(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)
        self.fake_A_sample = tf.compat.v1.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.input_c_dim], name='fake_A_sample')
        self.fake_B_sample = tf.compat.v1.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.output_c_dim], name='fake_B_sample')
        self.DB_real = self.discriminator(self.real_B, self.options, reuse=True, name="discriminatorB")
        self.DA_real = self.discriminator(self.real_A, self.options, reuse=True, name="discriminatorA")
        self.DB_fake_sample = self.discriminator(self.fake_B_sample, self.options, reuse=True, name="discriminatorB")
        self.DA_fake_sample = self.discriminator(self.fake_A_sample, self.options, reuse=True, name="discriminatorA")
        self.db_loss_real = self.criterionGAN(self.DB_real, tf.ones_like(self.DB_real))
        self.db_loss_fake = self.criterionGAN(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2
        self.da_loss_real = self.criterionGAN(self.DA_real, tf.ones_like(self.DA_real))
        self.da_loss_fake = self.criterionGAN(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2
        self.d_loss = self.da_loss + self.db_loss
        
        self.g_loss_a2b_sum = tf.compat.v1.summary.scalar("g_loss_a2b", self.g_loss_a2b)
        self.g_loss_b2a_sum = tf.compat.v1.summary.scalar("g_loss_b2a", self.g_loss_b2a)
        self.g_loss_sum = tf.compat.v1.summary.scalar("g_loss", self.g_loss)

        self.g_sum = tf.compat.v1.summary.merge([self.g_loss_a2b_sum, self.g_loss_b2a_sum, self.g_loss_sum])
        
        self.da_loss_sum = tf.compat.v1.summary.scalar("da_loss", self.da_loss)
        self.db_loss_sum = tf.compat.v1.summary.scalar("db_loss", self.db_loss)
        self.d_loss_sum = tf.compat.v1.summary.scalar("d_loss", self.d_loss)
        self.da_loss_real_sum = tf.compat.v1.summary.scalar("da_loss_real", self.da_loss_real)
        self.db_loss_real_sum = tf.compat.v1.summary.scalar("db_loss_real", self.db_loss_real)
        self.da_loss_fake_sum = tf.compat.v1.summary.scalar("da_loss_fake", self.da_loss_fake)
        self.db_loss_fake_sum = tf.compat.v1.summary.scalar("db_loss_fake", self.db_loss_fake)
        self.d_sum = tf.compat.v1.summary.merge(
            [self.da_loss_sum, self.da_loss_real_sum, self.da_loss_fake_sum,
             self.db_loss_sum, self.db_loss_real_sum, self.db_loss_fake_sum,
             self.d_loss_sum]
        )

        self.test_A = tf.compat.v1.placeholder(tf.float32,
                                     [None, self.image_size, self.image_size,
                                      self.input_c_dim], name='test_A')
        self.test_B = tf.compat.v1.placeholder(tf.float32,
                                     [None, self.image_size, self.image_size,
                                      self.output_c_dim], name='test_B')
        self.testA = self.generator(self.test_B, self.options, True, name="generatorB2A")
        self.testB = self.generator(self.test_A, self.options, True, name="generatorA2B")
        
        t_vars = tf.compat.v1.trainable_variables()
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]

beta1 is momentum term of Adam

In [25]:
class cyclegan(cyclegan):
  def sample_model(self, sample_dir, epoch, idx):
      dataA = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testA'))
      dataB = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testB'))
      np.random.shuffle(dataA)
      np.random.shuffle(dataB)
      batch_files = list(zip(dataA[:self.batch_size], dataB[:self.batch_size]))
      sample_images = [load_train_data(batch_file, is_testing=True) for batch_file in batch_files]
      sample_images = np.array(sample_images).astype(np.float32)

      fake_A, fake_B = self.sess.run(
          [self.fake_A, self.fake_B],
          feed_dict={self.real_data: sample_images}
      )
      save_images(fake_A, [self.batch_size, 1],
                  './{}/A_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))
      save_images(fake_B, [self.batch_size, 1],
                  './{}/B_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))

In [26]:
class cyclegan(cyclegan):
  def save(self, checkpoint_dir, step):
          model_name = "cyclegan.model"
          model_dir = "%s_%s" % (self.dataset_dir, self.image_size)
          checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

          if not os.path.exists(checkpoint_dir):
              os.makedirs(checkpoint_dir)

          self.saver.save(self.sess,
                          os.path.join(checkpoint_dir, model_name),
                          global_step=step)

In [27]:
class cyclegan(cyclegan):
  def load(self, checkpoint_dir):
        print(" [*] Reading checkpoint...")

        model_dir = "%s_%s" % (self.dataset_dir, self.image_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            print(f'check: {ckpt_name}')
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False

In [28]:
class cyclegan(cyclegan):
    def train(self, name='train'):
        self.lr = tf.compat.v1.placeholder(tf.float32, None, name='learning_rate')
        with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
            self.d_optim = tf.compat.v1.train.AdamOptimizer(self.lr, beta1=0.5) \
            .minimize(self.d_loss, var_list=self.d_vars)
            self.g_optim = tf.compat.v1.train.AdamOptimizer(self.lr, beta1=0.5) \
            .minimize(self.g_loss, var_list=self.g_vars)
        init_op = tf.compat.v1.global_variables_initializer()
        self.sess.run(init_op)
        self.writer = tf.compat.v1.summary.FileWriter("./logs", self.sess.graph)
        
        counter = 1
        start_time = time.time()
        
        if CONTINUE_TRAIN:
            if self.load(CHECKPOINT_DIR):
                print(" [*] Load SUCCESS")
            else:
                print(" [!] Load failed...")

        for epoch in range(1):
            dataA = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/trainA'))[100:150]
            dataB = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/trainB'))[100:150]
            np.random.shuffle(dataA)
            np.random.shuffle(dataB)
            batch_idxs = min(min(len(dataA), len(dataB)), 1e8) // self.batch_size
            lr = LR if epoch < EPOCH_STEP else LR*(200-epoch)/(200-EPOCH_STEP)
            
            for idx in range(batch_idxs):
                batch_files = list(zip(dataA[idx * self.batch_size:(idx + 1) * self.batch_size],
                                       dataB[idx * self.batch_size:(idx + 1) * self.batch_size]))
                batch_images = [load_train_data(batch_file, LOAD_SIZE, self.image_size) for batch_file in batch_files]
                batch_images = np.array(batch_images).astype(np.float32)
                
                # Update G network and record fake outputs
                fake_A, fake_B, _, summary_str = self.sess.run(
                    [self.fake_A, self.fake_B, self.g_optim, self.g_sum],
                    feed_dict={self.real_data: batch_images, self.lr: lr})
                self.writer.add_summary(summary_str, counter)
                [fake_A, fake_B] = self.pool([fake_A, fake_B])
                # Update D network
                _, summary_str = self.sess.run(
                    [self.d_optim, self.d_sum],
                    feed_dict={self.real_data: batch_images,
                               self.fake_A_sample: fake_A,
                               self.fake_B_sample: fake_B,
                               self.lr: lr})
                self.writer.add_summary(summary_str, counter)

                counter += 1
                print(("Epoch: [%2d] [%4d/%4d] time: %4.4f" % (
                    epoch, idx, batch_idxs, time.time() - start_time)))
                
                if np.mod(counter, PRINT_FREQ) == 1:
                    self.sample_model(SAMPLE_DIR, epoch, idx)

                # if np.mod(counter, args.save_freq) == 2:
                self.save(CHECKPOINT_DIR, counter)


In [29]:
class cyclegan(cyclegan):
  def test(self):
    init_op = tf.compat.v1.global_variables_initializer()
    self.sess.run(init_op)
    sample_files = glob('./datasets/{}/*.*'.format(self.dataset_dir + '/testA'))

    if self.load(CHECKPOINT_DIR):
        print(" [*] Load SUCCESS")
    else:
        print(" [!] Load failed...")

    index_path = os.path.join(TEST_DIR, '{0}_index.html'.format(WHICH_DIRECTION))
    index = open(index_path, "w")
    index.write("<html><body><table><tr>")
    index.write("<th>name</th><th>input</th><th>output</th></tr>")

    in_var, out_var = (self.test_A, self.testB)

    for sample_file in sample_files:
            print('Processing image: ' + sample_file)
            sample_image = [load_test_data(sample_file, FINE_SIZE)]
            sample_image = np.array(sample_image).astype(np.float32)
            image_path = os.path.join(TEST_DIR,
                                      '{0}_{1}'.format(WHICH_DIRECTION, os.path.basename(sample_file)))
            fake_img = self.sess.run(out_var, feed_dict={in_var: sample_image})
            save_images(fake_img, [1, 1], image_path)
            index.write("<td>%s</td>" % os.path.basename(image_path))
            index.write("<td><img src='%s'></td>" % (sample_file if os.path.isabs(sample_file) else (
                '..' + os.path.sep + sample_file)))
            index.write("<td><img src='%s'></td>" % (image_path if os.path.isabs(image_path) else (
                '..' + os.path.sep + image_path)))
            index.write("</tr>")
    index.close()

In [30]:
tfconfig = tf.compat.v1.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=tfconfig) as sess:
    model = cyclegan(sess)
    model.train()
    model.test()

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
 [*] Reading checkpoint...
check: cyclegan.model-51
INFO:tensorflow:Restoring parameters from ./checkpoint/med-image_256/cyclegan.model-51
 [*] Load SUCCESS
Epoch: [ 0] [   0/  50] time: 13.5681
Epoch: [ 0] [   1/  50] time: 24.2483
Epoch: [ 0] [   2/  50] time: 34.3776
Epoch: [ 0] [   3/  50] time: 44.4823
Epoch: [ 0] [   4/  50] time: 54.4077
Epoch: [ 0] [   5/  50] time: 64.2137
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: [ 0] [   6/  50] time: 74.1341
Epoch: [ 0] [   7/  50] time: 84.0363
Epoch: [ 0] [   8/  50] time: 93.8088
Epoch: [ 0] [   9/  50] time: 103.7435
Epoch: [ 0] [  10/  50] time: 113.6216
Epoch: [ 0] [  11/  50] time: 123.3981
Epoch: [ 0] [  12/  50] time: 133.5181
Epoch: [ 0] [  13/  50] time: 143.5890
Epoch: [ 0] [  14/  50] time: 153

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Processing image: ./datasets/med-image/testA/A_TB_1334_90284.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Processing image: ./datasets/med-image/testA/A_TB_1373_103148.jpg


...

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Processing image: ./datasets/med-image/testA/A_TB_1399_88670.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Processing image: ./datasets/med-image/testA/A_TB_1392_112105.jpg


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
